In [ ]:
!pip install pyspark

In [ ]:
!pip install -U pytz

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime,timedelta
import pytz

spark = SparkSession.builder\
        .appName("Test_unixTime")\
        .getOrCreate()

In [ ]:
spark

In [ ]:
tz_mexico = pytz.timezone('America/Mexico_City')
fecha_hoy = datetime.now(tz_mexico).strftime('%Y-%m-%d %H:%M:%S')
fecha_hoy_dt = datetime.now(tz_mexico)

In [ ]:
df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/test1.csv', header=True)

In [ ]:
df.show()

In [ ]:
df.count()

In [ ]:
df.select('name','age').show()

In [ ]:
df.printSchema()

In [ ]:
df.withColumn('Age',df['age'].cast('Integer')).printSchema()

In [ ]:
df.show()

In [ ]:
print(fecha_hoy)

In [ ]:
dff = df.withColumn('Fecha', lit(fecha_hoy))
dff.show()

In [ ]:
dff.printSchema()

In [ ]:
df_unix = dff.select('Name','age','Experience','Salary',unix_timestamp(col('Fecha')).alias('fecha'))
df_unix.show()

In [ ]:
df_not_unix = df_unix.select('Name','age','Experience','Salary',from_unixtime(col('Fecha')).alias('fecha'))
df_not_unix.show()

In [ ]:
df_convert = df_not_unix.withColumn("fecha", unix_timestamp(col('fecha')).cast('bigint'))
df_convert.show()
df_convert.printSchema()

In [ ]:
df_convert_2 = df_convert.withColumn('fecha', from_unixtime(col('fecha')).cast('string'))
df_convert_2.show()
df_convert_2.printSchema()

In [ ]:
df.select(count('Name')).collect()